## 부동산통계정보R-ONE

In [ ]:
import requests
import pandas as pd

"""
"CLS_FULLNM"= 지역, 
"WRTTIME_DESC"= 날짜, 
"DTA_VAL"= 매매수급지수
"""

# 한국부동산원 R-ONE API
url = "https://www.reb.or.kr/r-one/openapi/SttsApiTblData.do"

params = {
    "STATBL_ID": "A_2024_00076",  # 매매수급지수
    "KEY": "cc47bc2e1d1a4836b563bdd1226a9af8",
    "Type": "json",
    "DTACYCLE_CD": "MM",
    "WRTTIME_IDTFR_ID": "202410",
    "pIndex": 1,
    "pSize": 1000,
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status: {response.status_code}")

if data.get("SttsApiTblData") and len(data["SttsApiTblData"]) > 1:
    # 실제 데이터는 row에 있음
    df = pd.DataFrame(data["SttsApiTblData"][1]["row"])

    print(f"✓ {len(df)}개 데이터 조회 완료")
    print(f"\n컬럼: {df.columns.tolist()}")
    print(f"\n샘플 데이터:")
    print(df[["CLS_FULLNM", "WRTTIME_DESC", "DTA_VAL"]].head(10))
else:
    print(f"에러: {data}")

Status: 200
✓ 41개 데이터 조회 완료

컬럼: ['STATBL_ID', 'DTACYCLE_CD', 'WRTTIME_IDTFR_ID', 'GRP_ID', 'GRP_NM', 'CLS_ID', 'CLS_NM', 'ITM_ID', 'ITM_NM', 'DTA_VAL', 'UI_NM', 'GRP_FULLNM', 'CLS_FULLNM', 'ITM_FULLNM', 'WRTTIME_DESC']

샘플 데이터:
    CLS_FULLNM WRTTIME_DESC     DTA_VAL
0      서울>강북지역    2024년 10월  100.354213
1           전국    2024년 10월   94.061141
2  서울>강남지역>서남권    2024년 10월  101.897106
3  서울>강북지역>도심권    2024년 10월  101.617647
4      부산>중부산권    2024년 10월   87.000000
5      경기>경부1권    2024년 10월  101.517067
6  서울>강남지역>동남권    2024년 10월  101.118476
7          수도권    2024년 10월   98.133966
8  서울>강북지역>동북권    2024년 10월   99.425947
9      부산>동부산권    2024년 10월   85.887097


>는 계층 구조 구분자입니다!
```
📍 계층 구조
서울 > 강남지역 > 서남권
 ↓       ↓         ↓
광역시   권역      세부권역
예시로 보면
"서울 > 강남지역 > 서남권"
  └─ 서울시
      └─ 강남지역 (강남, 서초, 송파, 강동 등)
          └─ 서남권 (강남구, 서초구 일부)

"부산 > 중부산권"
  └─ 부산광역시
      └─ 중부산권 (중구, 서구 등)
```

## 매매수급지수 조회 방법

### 1️⃣ STATBL_ID 찾기
**통계코드 검색**: https://www.reb.or.kr/r-one/portal/openapi/openApiGuideCdPage.do
- "매매수급지수" 검색 → STATBL_ID 확인

### 2️⃣ 인증키 발급
**R-ONE 로그인**: https://www.reb.or.kr/r-one/
- 로그인 → Open API → 인증키 발급

### 3️⃣ API 파라미터
- `STATBL_ID`: 통계표 ID
- `DTACYCLE_CD`: `MM`(월별), `YY`(연별)
- `WRTTIME_IDTFR_ID`: 조회 시점 (예: `202410`)
- `KEY`: 발급받은 인증키

In [ ]:
def get_supply_demand_index(address):
    """
    주소로 매매수급지수 조회 (개선 버전)
    - 모든 유형의 주소 지원
    - 부분 매칭으로 유사한 위치도 조회
    """
    from difflib import get_close_matches
    import re

    # 1단계: 입력값 정제
    address_clean = address.strip()
    if not address_clean:
        return "주소를 입력해주세요 (예: 서울, 강남, 부산 해운대구)"

    # 2단계: 모든 단어를 키워드로 추출 (공백과 특수문자 기준)
    keywords = re.split(r"[\s,>]+", address_clean)
    keywords = [k for k in keywords if len(k) > 0]

    if not keywords:
        return "유효한 주소를 입력해주세요"

    # 3단계: 정확한 매칭 시도 (OR 조건)
    pattern = "|".join(keywords)
    result = df[df["CLS_FULLNM"].str.contains(pattern, case=False, na=False)]

    # 4단계: 결과가 없으면 유사도 기반 검색
    if len(result) == 0:
        all_locations = df["CLS_FULLNM"].unique().tolist()

        # 각 키워드로 유사한 지역명 찾기
        similar_matches = []
        for keyword in keywords:
            matches = get_close_matches(keyword, all_locations, n=5, cutoff=0.6)
            similar_matches.extend(matches)

        if similar_matches:
            # 유사한 지역들 조회
            result = df[df["CLS_FULLNM"].isin(similar_matches)]

    # 5단계: 결과 반환
    if len(result) > 0:
        result_clean = result[["CLS_FULLNM", "WRTTIME_DESC", "DTA_VAL"]].copy()
        result_clean.columns = ["지역", "날짜", "매매수급지수"]

        # 관련성 높은 순으로 정렬 (입력 키워드와 많이 매칭되는 순)
        def match_score(location):
            return sum(1 for k in keywords if k in location)

        result_clean["_score"] = result_clean["지역"].apply(match_score)
        result_clean = result_clean.sort_values("_score", ascending=False).drop(
            "_score", axis=1
        )

        return result_clean.reset_index(drop=True)
    else:
        # 사용 가능한 지역 일부 안내
        available = df["CLS_FULLNM"].unique()[:10].tolist()
        return (
            f"'{address}'에 해당하는 데이터가 없습니다.\n\n사용 가능한 지역 예시:\n"
            + "\n".join(f"- {loc}" for loc in available)
        )


# 사용 예시
print("🔍 테스트 1: 부산 중부산")
print(get_supply_demand_index("부산 중부산"))
print("\n" + "=" * 60 + "\n")

print("🔍 테스트 2: 부산 (단일 키워드)")
print(get_supply_demand_index("부산"))
print("\n" + "=" * 60 + "\n")

print("🔍 테스트 3: 도심권")
print(get_supply_demand_index("도심권"))
print("\n" + "=" * 60 + "\n")

print("🔍 테스트 4: 강남")
print(get_supply_demand_index("강남"))
print("\n" + "=" * 60 + "\n")

print("🔍 테스트 5: 전북 전주시 완산구 서신동 비사벌아파트 101동 701호")
print(get_supply_demand_index("서울 강남"))

🔍 테스트 1: 부산 중부산
        지역         날짜     매매수급지수
0  부산>중부산권  2024년 10월  87.000000
1  부산>동부산권  2024년 10월  85.887097
2  부산>서부산권  2024년 10월  70.250000
3       부산  2024년 10월  81.902058


🔍 테스트 2: 부산 (단일 키워드)
        지역         날짜     매매수급지수
0  부산>중부산권  2024년 10월  87.000000
1  부산>동부산권  2024년 10월  85.887097
2  부산>서부산권  2024년 10월  70.250000
3       부산  2024년 10월  81.902058


🔍 테스트 3: 도심권
            지역         날짜      매매수급지수
0  서울>강북지역>도심권  2024년 10월  101.617647


🔍 테스트 4: 강남
            지역         날짜      매매수급지수
0  서울>강남지역>서남권  2024년 10월  101.897106
1  서울>강남지역>동남권  2024년 10월  101.118476
2      서울>강남지역  2024년 10월  101.556843


🔍 테스트 5: 전북 전주시 완산구 서신동 비사벌아파트 101동 701호호
            지역         날짜      매매수급지수
0  서울>강남지역>서남권  2024년 10월  101.897106
1  서울>강남지역>동남권  2024년 10월  101.118476
2      서울>강남지역  2024년 10월  101.556843
3      서울>강북지역  2024년 10월  100.354213
4  서울>강북지역>도심권  2024년 10월  101.617647
5  서울>강북지역>동북권  2024년 10월   99.425947
6  서울>강북지역>서북권  2024년 10월  102.333333
7           서울  2024년 10월 

: 